# Introduction to Functional Programming

## Outcomes

- understand what functional programming is,
- know how to use `map`, `filter`, `reduce`,
- understand what lazy computation & lambda functions are.


## Why learn about functional programming in a course on distributed computation?

Functional programming concepts (such as map, filter, and reduce) are the atomic operations of many distributed computing frameworks - the best example being MapReduce.

An example we will become familar with is **mapping over a pool of processes**:

```python
from multiprocessing import Pool

with Pool(popsize, maxtasksperchild=32) as pool:
    results = p.starmap(episode, zip(population, seeds))
```


# What is Functional Programming?

Functional programming ideals/goals/principles:

1. no side effects,
2. variables don't vary,
3. first class functions.

Expressing a program with pure functions avoids internal state & multiple entry points - things that make programming harder ([John Carmack](http://sevangelatos.com/john-carmack-on-parallel-implementations/)).

Functional programming also encourages functional decomposition - structuring a program with functions to reduce code re-use & improve readability.


## Context with other programming languages

Many programming languages are **procedural** & **imperative**:

- say both what you want and how to do it,
- specify the problem to be solved & how to solve it,
- explicitly program & control how a program runs (loops, variables etc),
- examples include Python, Javascript and C.

Another approach is **declarative**:

- say what you want - not how,
- specify the problem to be solved,
- do not specify how to solve it,
- examples include HTML & SQL.

**Functional programming is declarative** - it is about **what you want**.  The mechanics of the how (such as sequential or parallel execution) can be hidden in a functional program (see `pool.map`).


## 1. No Side Effects

Things always run / perform the same way:

- no dependency on the state of the outside world,
- nothing can happen to make our program run differently,
- same inputs = same outputs.

This is **idempotency** - that your Nth time running a function should return the same thing as the first time.  

No matter how many times you call the function, the result is the same.

Same inputs always give the same outputs.

### Example - two ways to close a door

We can close a door using a toggle:

In [1]:
import dataclasses

@dataclasses.dataclass
class Door:
    position: str

def toggle(door: Door):
    if door.position == 'open':
        door.position = 'closed'
        
    elif door.position == 'closed':
        door.position = 'open'
        
    else:
        pass
        
    return door
            
door = Door('open')
for _ in range(5):
    door = toggle(door)
    print(door)

Door(position='closed')
Door(position='open')
Door(position='closed')
Door(position='open')
Door(position='closed')


Closing a door with a toggle is not idempotent - the result of our action is different depending on state held in the door position.

Another way to close a door is to always lock - which is idempotent:

In [2]:
def close(door: Door):
    if door.position == 'open':
        door.position = 'closed'
    return door
            
door = Door('open')
for _ in range(5):
    door = close(door)
    print(door)

Door(position='closed')
Door(position='closed')
Door(position='closed')
Door(position='closed')
Door(position='closed')


This second, idempontent version of the program no longer depends on the state of the door. 

The `close` action will always result in a closed door.

## 2. Variables don't vary

Variables are only ever initialized - they are never changed.  All data is immutable.

We can demonstrate the inverse of this principle by showing how data mutation causes a problem:

In [3]:
def pipeline(data):
    data['stage'] = 'processed'
    return data

raw = {'stage': 'raw', 'pkg': 'some-data'}
processed = pipeline(raw)

#  our data processing has corrupted & mutated our raw data
print(raw)

{'stage': 'processed', 'pkg': 'some-data'}


In [4]:
def immutable_pipeline(data):
    processed = {
        'stage': 'processed',
        'pkg': data['pkg']
    }
    return processed

raw = {'stage': 'raw', 'pkg': 'some-data'}
processed = immutable_pipeline(raw)

#  here our raw data remains the same - immutable
print(raw)

{'stage': 'raw', 'pkg': 'some-data'}


In the age of cheap storage in data lakes, taking the approach of saving data (ie state) in between each stage of a data pipeline is economic.

## 3. First class functions

This is a feature of programming languages - a language with first class functions is one where functions can be passed around as objects:

In [5]:
def controller(func, data):
    return func(data)

data = [1, 2, 3]
print(controller(sum, data), controller(len, data))

6 3


# Map, Lambda Functions & Lazy Computation

Mapping takes an iterable and apply a function to each element.

A map operation is functionally the same as a `for` loop, or `df.apply` in `pandas`.

We can run a function on each element of a list using a `for` loop:

In [6]:
def lower(s):
    return s.lower()

cities = ['Berlin', 'Auckland', 'London', 'Christchurch']
[lower(s) for s in cities]

['berlin', 'auckland', 'london', 'christchurch']

We can achieve the same result with the Python built-in `map`.

Calling `map` with our function (that we want to apply to each element) and iterable returns a `map` generator - an example of **lazy computation**:

In [7]:
lazy = map(lower, cities)
lazy

As we are more impatient than lazy, we can get all the processed data by calling `list` on our generator:

In [8]:
list(lazy)

['berlin', 'auckland', 'london', 'christchurch']

## Lambda functions

Lambda functions are anonymous - they are functions without a name - there is no variable that refers to the function.

Lambda functions are built on the fly - there is no functional difference between a defining the function or use an anonymous function (same as no function difference between `map` & `for` loop).

We can rewrite our data pipeline to use a lambda function - we still pass in our function & iterable, just this time our function is anonymous:

In [9]:
list(map(lambda x: x.lower(), cities))

['berlin', 'auckland', 'london', 'christchurch']

In a lambda function we define both the structure of the input, functionality & output:

In [10]:
#  lambda that just returns input
lambda x: x

#  lambda that just returns input
lambda name: name 

#  lambda that takes two inputs (ie a tuple) and sums them
lambda x, y: x + y

#  lambda that takes a dict as input:
lambda x: x['key']

<function __main__.<lambda>(x)>

We can also do more complex things in lambda functions, such as accessing elements of our input data.

The pipeline below takes a tuple as input & accesses certain elements to perform computation on:

In [11]:
from collections import namedtuple
from helpers import get_populations

populations = get_populations()

list(map(lambda x: (x[0], x[1] * 1000), populations))

[('Berlin', 3700.0),
 ('Auckland', 1700.0),
 ('London', 8900.0),
 ('Sheffield', 500.0),
 ('Christchurch', 380.0)]

When using `map` we have total flexibility around **what data structure we use as input** and **how we interact with it** in the function.

# Filter

Tests each element, keeps those that pass:

In [12]:
list(filter(lambda x: x[1] > 1.0, populations))

[city(city='Berlin', population=3.7, continent='eu'),
 city(city='Auckland', population=1.7, continent='pac'),
 city(city='London', population=8.9, continent='eu')]

Equilivant program without `filter`:

In [13]:
[city for city in populations if city[1] > 1.0]

[city(city='Berlin', population=3.7, continent='eu'),
 city(city='Auckland', population=1.7, continent='pac'),
 city(city='London', population=8.9, continent='eu')]

# Reduce


Reduce is a form of data aggregation - aggregating data to a single value (or single values per group).

For example, if we want to calculate the total populations of all our cities:

In [14]:
sum([city.population for city in populations])

15.180000000000001

Another model for this computation (which is more similar to how `reduce` works) works by initializing a counter `total`.

This counter `total` is is internal state passed around each time we iterate over our input data:

In [15]:
total = 0
for p in populations:
    total += p.population
total

15.180000000000001

Reduce is NOT a Python built-in function - it's hidden away in `functools`.

Using `reduce` is similar to `map` - we pass in a function & iterable - but with `reduce` we also pass in an initial accumulator value.

This initial accumulator can be any data type - the flexibility to set the initial accumulator makes `reduce` flexible:

In [16]:
reduce?

Object `reduce` not found.


In [17]:
from functools import reduce

def sum_population(total, city):
    return total + city.population

reduce(sum_population, populations, 0)

15.180000000000001

In [18]:
reduce(lambda total, pop: total + pop[1], populations, 0)

15.180000000000001

We have complete control over how we iterate & aggregate.

For example, we can perform a groupby - here just selecting cities by region:

In [19]:
def group(acc, city):
    acc[city.continent].append(city.city)
    return acc

reduce(
    group,
    populations,
    {'eu': [], 'pac': []}
)

{'eu': ['Berlin', 'London', 'Sheffield'], 'pac': ['Auckland', 'Christchurch']}

# Exercises



Now it's your turn ^^

Advice:

- don't be afraid to first write the program in a non-functional style,
- can be eaiser to convert an existing program to a functional style than create from scratch,
- answers are in `src/answers.py`.

## Exercise

The code below sums the total population for the `pac` continent:

In [20]:
from helpers import get_populations

populations = get_populations()

output = []
for city in populations:
    if city.continent == 'pac':
        output.append(city.population)
        
total_population = sum(output)
print(total_population)

2.08


Convert this code to use `map`, `filter` & `itertools.reduce`:

## Exercise

Create a data processing pipeline that selects the cities that have populations greater than the average of all cities.

Two steps:

1. calculate the average city population (with our without reduce),
2. use a filter to select.

Calculating the average via a reduce can be done via incremental mean updating.

## Exercise

Create a data processing pipeline that finds the average population for both continents.

Two steps:

1. reduce to (key, populations),
2. map to (key, avg).

# Further Reading / References

[Functional Programming HOWTO](https://docs.python.org/3/howto/functional.html)